In [1]:
import os
from src.utils.utils import connect_to_mongoDB
from dotenv import load_dotenv
import pandas as pd
import pymongo
from pymongo.write_concern import WriteConcern

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
load_dotenv(".env")
MONGODB_USER = os.getenv("MONGODB_USER")
MONGODB_PWD = os.getenv("MONGODB_PWD")

client = connect_to_mongoDB(MONGODB_USER, MONGODB_PWD)
ctgov = client["ctGov"]
asthma = ctgov["asthma"]
hf = ctgov["heart_failure"]
preprocessed = ctgov["preprocessed"]

# preprocessed.delete_many({})
# preprocessed.with_options(write_concern=WriteConcern(w=0)).insert_many(hf.find({}), ordered=False)
# preprocessed.with_options(write_concern=WriteConcern(w=0)).insert_many(asthma.find({}), ordered=False)

c:\Users\joanp\Projects\ctgov_rag\.venv\Lib\site-packages\pymongo\uri_parser.py:321: UserWarning: The value of retrywrites must be 'true' or 'false'
  return get_validated_options(opts, warn)


Pinged your deployment. You successfully connected to MongoDB!


In [4]:
# for sheet in ["protocolSection", "resultsSection", "annotationSection", "documentSection", "derivedSection",]:
#     df = pd.read_excel("./docs/ctGov.metadata.xlsx", sheet)
    
#     for idx in df.loc[df["Used"]=="N", "Index Field"].values:
#         preprocessed.update_many({}, {'$unset':{idx:1}})
# preprocessed.update_many({}, {'$unset':{"trial2vec":1}})

In [5]:
import nest_asyncio
nest_asyncio.apply()

In [6]:
from langchain_community.document_loaders.mongodb import MongodbLoader

app_name =  "cluster0"

loader = MongodbLoader(
    connection_string=f'mongodb+srv://{MONGODB_USER}:{MONGODB_PWD}@{app_name}.bcn2gwy.mongodb.net/?retryWrites="TRUE"&w=majority&appName={app_name.capitalize()}',
    db_name="ctGov",
    collection_name="preprocessed",
)

c:\Users\joanp\Projects\ctgov_rag\.venv\Lib\site-packages\pymongo\uri_parser.py:321: UserWarning: The value of retrywrites must be 'true' or 'false'
  return get_validated_options(opts, warn)


In [7]:
docs = loader.load()

In [8]:
for i in range(len(docs)):
    docs[i].page_content = docs[i].page_content.replace("\'", "")
    docs[i].metadata["filename"] = docs[6].page_content[6:17] 
    

In [9]:
docs[:10]

[Document(page_content='{_id: NCT00000475, protocolSection: {identificationModule: {nctId: NCT00000475, organization: {fullName: National Heart, Lung, and Blood Institute (NHLBI), class: NIH}, briefTitle: Prevention of Early Readmission in Elderly Congestive Heart Failure Patients}, statusModule: {overallStatus: COMPLETED}, sponsorCollaboratorsModule: {leadSponsor: {name: National Heart, Lung, and Blood Institute (NHLBI), class: NIH}}, descriptionModule: {briefSummary: To assess the impact of a multidisciplinary treatment program on three-month readmission-free survival in elderly congestive heart failure patients., detailedDescription: BACKGROUND:\\n\\nThe prevalence of congestive heart failure increases progressively with advancing age, and congestive heart failure is a major cause of mortality, morbidity, and diminished quality of life in the elderly. Not only is congestive heart failure the most common cardiovascular indication for hospitalization in patients 65 years or older, but

In [10]:
from langchain_text_splitters import RecursiveJsonSplitter
splitter = RecursiveJsonSplitter(max_chunk_size=300)

In [11]:
studies = []
for study in preprocessed.find({}):
    nctId = study.pop("_id")
    studies.append({nctId:study})

In [13]:
json_docs = splitter.create_documents(texts=studies)

In [ ]:
json_docs[:10]